# Simulating quantum error correction

## Contents

- [Introduction](#intro)
    - [Stabilizer quantum error correcting codes](#stabilizer)
    - [Error correction in stabilizer codes](#decoding)
- [Simulating stabilizer error correction](#stabilizer_simulation)
    - [The qecsim software package](#qecsim)
    - [The stim software package](#stim)
- [Example: error correction in the toric code](#example1)
    - [Toric code stabilizers and logicals](#tc)
    - [Errors and corrections](#tc_qec)
    - [The toric code error correction threshold](#tc_threshold)
- [Example: adding circuit and measurement noise](#example2)
    - [Implementing error correcting codes](#circuits)
    - [Adding gate and measurement noise](#noise)
    - [Fault-tolerant threshold](#fault_tolerant)

## Introduction <a id='intro'></a>

### Stabilizer quantum error correcting codes <a id='stabilizer'></a>

Recall the definition of a stabilizer quantum error correcting code from the lectures. A stabilizer code is defined by an Abelian subgroup $\mathcal{S}$ of the $n$-qubit Puli group, whose joint $+1$-eigenspace gives the codespace. We usually specify $\mathcal{S}$ in terms of a set of generators
$$\{g_1,g_2\ldots g_t\}$$
such that any other operator in $\mathcal{S}$ can be written as a product of the $g_i$'s.

The logical operators of a stabilizer code are operators that act within the code space, meaning they must map stabilizers to stabilizers under conjugation. In particular, logical Pauli operators are Pauli operators that commute with all elements of $\mathcal{S}$. Logical Pauli operators are not unique, but are only defined up to multiplication by elements of $\mathcal{S}$. If $P$ is a logical Pauli operator, then $Ps = sP$ is also a logical operator with the same action on the code spae, for all $s \in \mathcal{S}$.

### Error correction in stabilizer codes <a id='decoding'></a>

Error correction in stabilizer codes is performed by repeatedly measuring a complete set of stabilizer generators. When an error occurs on the code space, any stabilizer whose eigenvalue is not measured to be $+1$ is said to belong to the syndrome of that error. Based on that syndrome, a decoding algorithms then decides on a suitable correction operator to be applied that brings the system back to the code space, and, hopefully, should cancel the error. If the product of this correction and the original error lies in the stabilizer group, then the total action is trivial and error correction is successful. If the product is not in the stabilizer group, then a logical error has occurred, and error correction has failed.

For our discussion we will restrict to Pauli errors $E$, which by definition can only commute or anticommute with the stabilizer generators. The syndrome of a Pauli errors is then given by the generators that anticommute with the error. Based on the syndrome the decoder proposes a recovery $E$ such that $ER$ commutes with the stabilizer, in the hope that this product actually lies in the stabilizer.

## Simulating stabilizer error correction <a id='stabilizer_simulation'></a>

There exist many packages that provide tools for simulating quantum error correction. Here we will focus on just two of them, which are particularly suitable for pedagogical purposes. Each of these packages tackles the problem of error correction from a slightly different angle. We will start by considering stabilizer error correction in a more 'traditional' sense, focusing on stabilizers and logical operators. Next, we will consider a more 'circuit-based' approach, where the central focus lies on the quantum circuits that actually implement the stabilizer measurements, and how to sample these circuits under different conditions.

### The [qecsim](https://qecsim.github.io/index.html) software package <a id='qecsim'></a>

The first package we will use is qecsim.

<div class="alert alert-block alert-info"> <b>From the qecsim documentation:</b> <i>qecsim is a Python 3 package for simulating quantum error correction using stabilizer codes. It is lightweight, modular and extensible, allowing additional codes, error models and decoders to be plugged in.</i> </div>

An honourable mention for a package with similar functionalities goes to [PanQEC](https://panqec.readthedocs.io/en/latest/index.html). This latter package has some more builtin codes and decoders and is therefore very suitable for exploring stabilizer error correction. However, the simple ASCII visualizations exported by qecsim give it an edge for our current purpose. In that spirit, you are alsoencouraged to play around with the [exellent quantum code visualizer found here](https://gui.quantumcodes.io/).

### The [Stim](https://github.com/quantumlib/Stim/tree/main) software package <a id='stim'></a>

The next package we will use is Stim.

<div class="alert alert-block alert-info"> <b>From the Stim README:</b> <i>Stim is a tool for high performance simulation and analysis of quantum stabilizer circuits, especially quantum error correction (QEC) circuits.</i> </div>

Stim is a very powerful tool for analyzing and sampling stabilizer circuits under noise models of varying complexity. However, it does not directly contain decoders to decode the stabilizer measurement samples. A decoder that works particularly well in conjunction with Stim is [PyMatching](https://pymatching.readthedocs.io/en/stable/#), which is a very flexible Python implementation of a minimum-weight perfect matching decoder.

### Example: error correction in the toric code <a id='example1'></a>

As a first example, we will consider 'conventional' stabilizer error correction in the toric code under bit-flip noise using qecsim.

In [ ]:
from qecsim.models.toric import ToricCode
from qecsim_wrappers import print_code, print_pauli

### Toric code stabilizers and logicals <a id='tc'></a>

Recall that the toric code is a stabilizer code defined on a 2D square lattice with periodic boundary conditions, where a qubit is placed on each edge of the lattice. We will assume the lattice is of size $L\times L$, so there are $n=2L^2$ qubits.

The toric code stabilizer generators are given by the star and plaquette operators $ A_s $ and $ B_p $, defined as
$$A_s = \prod_{j \in s} Z_j\,, \qquad B_p = \prod_{j \in p} X_j\,,$$
where $ j \in s $ and $ j \in p $ denote all the edges $ j $ incident to the vertex $ s $, and bounding the plaquette $ p $ respectively.
The stabilizer group of the toric code is then
$$\mathcal{S} = \{ A_s, B_p \mid s \text{ is a vertex of the lattice},\, p \text{ is a plaquette of the lattice} \}.$$
The logical operators are given by non-contractible loops of $X$ and $Z$ operators on the lattice and dual lattice respectively.

qecsim implements the toric code as a specific subtype of a `StabilizerCode`, `ToricCode`. As a first step, we can construct a toric code object for a given lattice size and inspect its properties.

In [ ]:
L = 8 # linear lattice size
p = 0.05 # error probability

code = ToricCode(L, L)

We can inspect the code parameters:

In [ ]:
code.n_k_d # [[2L^2, 2, L]]

We can print the code layout to check the geometry:

In [ ]:
print_code(code)

We can print some stabilizer generators:

In [ ]:
len(code.stabilizers) # 2 L^2 stabilizers

In [ ]:
print_pauli(code, code.stabilizers[14]) # a random plaquette stabilizer

In [ ]:
print_pauli(code, code.stabilizers[90]) # a random vertex stabilizer

And we can print the pairs of logical operators:

In [ ]:
len(code.logicals) # 4 logical operators

In [ ]:
# the first pair of logical paulis
print_pauli(code, code.logicals[0]) # first logical Z
print_pauli(code, code.logicals[2]) # first logical X

In [ ]:
# the second pair of logical paulis
print_pauli(code, code.logicals[1]) # second logical Z
print_pauli(code, code.logicals[3]) # second logical X

### Errors and corrections <a id='tc_qec'></a>

Next, we can have a look at errors, syndromes, decoders, correction operators, and (potential) logical errors.

In [ ]:
from qecsim.models.toric import ToricMWPMDecoder
from qecsim_wrappers import bit_flip_model, compute_syndrome, print_syndrome, pauli_product, logical_commutations, effective_logical

We will use a simple bit-flip error model, which is an instance of a a subtype of [`qecsim.model.ErrorModel`](https://qecsim.github.io/api/model.html#qecsim.model.ErrorModel).

In [ ]:
error_model = bit_flip_model()

Given a random error model and a code, we can generate a random error by calling the [`generate` method](https://qecsim.github.io/api/model.html#qecsim.model.ErrorModel.generate) method by supplying it with the code and a physical error rate.

In [ ]:
p = 0.05 # physical error probability per qubit
E = error_model.generate(code, p)
print_pauli(code, E)

Once we have an error we can evaluate its syndrome using `compute_syndrome`. This returns a binary vector indicating the stabilizer that anticommute with the error.

In [ ]:
syndrome = compute_syndrome(code, E)
syndrome

We can also print this syndrome which shows that it, of course, only consists only of $Z$-type vertex stabilizers.

In [ ]:
print_syndrome(code, syndrome)

qecsim also provides a minimum-weight perfect matching decoder for the toric code, `ToricMWPMDecoder`, which we can use to correct our random error error. After initializing the decoder, we can pass the code and the error syndrom to its [][`decode` method](https://qecsim.github.io/api/model.html#qecsim.model.Decoder.decode) method to obtain a correction Pauli operator.

In [ ]:
decoder = ToricMWPMDecoder() # initializer MWPM decoder for the toric code
R = decoder.decode(code, syndrome)
print_pauli(code, R)

For low noise rates, it is easy to see that the correction operator pairs up neighboring vertex syndromes with strings of $Z$ errors. This is precisely what we expect from a matching decoder.

Computing the product of the error and the correction operator using [`pauli_prodcut`], we can visualize its combined action. For low noise rates, this will usually be trivial or clearly correspond to a stabilizer, indication successful error correction.

In [ ]:
ER = pauli_product(E, R)
print_pauli(code, ER)

To verify that it is indeed trivial, we can check if the total action $ER$ anticommutes with any of the logcical operators using the `logical_commutations` function. If it commutes with all logicals it is a logical identity and correction is successful. If it instead anticommutes with any of the logicals it is a logical operator itself, and correction fails.

In [ ]:
anticommutations = logical_commutations(code, ER)
anticommutations # returns a binary vector indicating which of the logicals the operator anticommutes with

As a convenience tool, we can use the function `effective_logical` to find the logical operator that corresponds to the total action of the error and correction.

In [ ]:
eff_log, eff_label = effective_logical(code, ER)
print(f"Effective logical operator after error and recovery:\t{eff_label}")
print_pauli(code, eff_log)

Try playing around with the physical error rate to see how the decoder performs at different noise levels. You will find more and more logical errors as the noise rate increases. Intuitively, you should be able to see how pairing up syndromes that are close to each other does not always work in these regimes.

To explicitly show the limitations of the matching decoder, we can have a look at two errors with the same syndrome, one of which is correctable and the other is not.

In [ ]:
from qecsim_wrappers import good_E, bad_E

In [ ]:
print_pauli(code, good_E) # weight-3 error, where 3 < d/2, so correctable

In [ ]:
good_s = compute_syndrome(code, good_E)
print_syndrome(code, good_s)

In [ ]:
successfull_R = decoder.decode(code, good_s)
print_pauli(code, successfull_R) # recover is the same as the original error

In [ ]:
eff_log, eff_label = effective_logical(code, pauli_product(good_E, successfull_R))
print(f"Effective logical operator after error and recovery:\t{eff_label}")
print_pauli(code, eff_log) # total effect is trivial

In [ ]:
print_pauli(code, bad_E)

In [ ]:
bad_s = compute_syndrome(code, bad_E)
print_syndrome(code, bad_s) # same syndrome

In [ ]:
unsuccessfull_R = decoder.decode(code, bad_s)
print_pauli(code, unsuccessfull_R) # but the recovery is now different from the error

In [ ]:
eff_log, eff_label = effective_logical(code, pauli_product(bad_E, unsuccessfull_R))
print(f"Effective logical operator after error and recovery:\t{eff_label}")
print_pauli(code, eff_log) # and their total action is a logical X

### The toric code error correction threshold <a id='tc_threshold'></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qecsim import app

The tools we have demonstrated so far are all we need to estimate the error correction threshold of the toric code. To do this, we simply need to run the error correction procedure outlined above many times and compute the logical failure rate as the total number of logical errors divided by the number of runs, and this for several values of the physical error rate and code size. We then hope to find a critical value $p_{\text{th}}$ of the physical error rate below which the logical failure rate goes to zero in the limit of large code size.

We will perform this analysis using the convenient [application functions in the qecsim `app` module](https://qecsim.github.io/api/app.html#).

In [ ]:
# initialize run parameters

# set models
codes = [ToricCode(*size) for size in [(3, 3), (5, 5), (7, 7), (9, 9)]]
error_model = bit_flip_model()
decoder = ToricMWPMDecoder()
# set physical error probabilities
error_probability_min, error_probability_max = 0.06, 0.14
error_probabilities = np.linspace(error_probability_min, error_probability_max, 5)
# set max_runs for each probability
max_runs = 1000

# print run parameters
print('Codes:', [code.label for code in codes])
print('Error probabilities:', error_probabilities)

In [ ]:
# run simulations and print data from middle run to view format; this might take a while
data = [app.run(code, error_model, decoder, error_probability, max_runs=max_runs)
        for code in codes for error_probability in error_probabilities]
print(data[len(data)//2])

In [ ]:
# prepare data for plotting
code_to_xys = {}
for run in data:
    xys = code_to_xys.setdefault(run['code'], [])
    xys.append((run['physical_error_rate'], run['logical_failure_rate']))

In [ ]:
# plot the simulated data
fig = plt.figure(1, figsize=(8, 5))
plt.title('Toric code simulation\n(Bit-flip noise, {} decoder)'.format(decoder.label))
plt.xlabel('Physical error rate')
plt.ylabel('Logical failure rate')
plt.xlim(error_probability_min-0.01, error_probability_max+0.01)
plt.ylim(-0.05, 0.7)
# add data
for code, xys in code_to_xys.items():
    plt.plot(*zip(*xys), 'x-', label='{} code'.format(code))
plt.legend(loc='lower right')
plt.show()

After some simulation time, we find a threshold of about $p_{\text{th}} \approx 10\%$.

## Example: adding circuit and measurement noise <a id='example2'></a>

### Implementing error correcting codes <a id='circuits'></a>

When putting error correction into practice on a physical device, we need a way of actually measuring the stabilizer generators. As we have seen in the exercise sessions, this can be done by introducing an ancilla qubit for each stabilizer generator, entangling these ancilla in a suitable way with the qubits their corresponding stabilizer acts on, and then measuring the ancilla qubits in the computateional basis.

Indeed, we have already seen some examples of these so-called *syndrome extraction circuits*. A $Z^{\otimes 4}$ stabilizer can be measured using the $Z$-parity circuit

<center><img src="./fig/z_stabilizer_circuit.svg" alt="z_stabilizer"/></center>

while an $X^{\otimes 4}$ stabilizer can be measured using the $X$-parity circuit

<center><img src="./fig/x_stabilizer_circuit.svg" alt="x_stabilizer"/></center>

However, once we acknowledge the fact that we constantly have to execute these kinds of syndrome extraction circuits and add to this the fact that any component and operation in an actual quantum device is noisy, it becomes clear that we should take into account the noise that occurs during the syndrome extraction process when simulating error correction.

To this end we will have a look at the [Stim software package](https://github.com/quantumlib/Stim/tree/main), which allows us to define error correcting codes directly from the circuits that are used to implement them.

In [ ]:
import stim

We'll start by looking at an example, namely the so-called rotated surface code, which you can think of as a version of the toric code implemented on a finite plane with open boundary conditions. The circuit that implements 4 rounds of syndrome measurements in a distance-3 surface code can be loaded as:

In [ ]:
circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_z",
    rounds=4,
    distance=3,
)

When we print it, we see it consists of a sequence of instructions written in Stim's own internal format:

In [ ]:
circuit

However, we can visualize the circuit on a time-line to get a clear picture of what it does:

In [ ]:
circuit.diagram('timeline-svg')

We immedialy see that the circuit implements a sequence of $X$ and $Z$ stabilizer measurements very similar to those of the toric code. We also note that it uses the measurement outcomes to define so-called 'detectors'. A detector is nothing more than a collection of measurement results that should sum to zero modulo 2. Here, the detectors simply encode that, in the absence of errors, the stabilizer measurements in any two consecutive rounds of syndrome extraction should have the same outcome.

Another nice way of visualizing the flow of entangling gates and ancilla measurements is to plot time slices of the circuit:

In [ ]:
circuit.diagram('timeslice-svg')

### Adding gate and measurement noise <a id='noise'></a>

The circuit we have just looked at is annotated with detectors, which detect whether stabilizer measurements between different rounds have the same outcome. Of course in an ideal world this would always be the case, but in reality noise can flip some of these outcomes, triggering certain detectors.

Moreover, in contrast to the simple bit-flip noise we have considered so far, noise in real devices is not limited to flipping qubits. Instead, we have to deal with noise that can occur at any point in the circuit, and that can be of many different types. For example, a gate might not be executed correctly, a measurement might be faulty, an ancilla reset may fail, and so on.

All these different kinds of noise pocesses can be added to a circuit:

In [ ]:
circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_z",
    rounds=4,
    distance=3,
    after_clifford_depolarization=0.001, # gate-level noise
    after_reset_flip_probability=0.001, # ancilla reset noise
    before_measure_flip_probability=0.001, # measurement noise
    before_round_data_depolarization=0.001, # data qubit noise
)

In [ ]:
circuit.diagram('timeline-svg')

In [ ]:
circuit.diagram('timeslice-svg')

Without going into the specifics of all these error processes, it is immediately clear that figuring out the effect of all these noise processes on the stabilizer measurements is a non-trivial task. This is where Stim comes in. Stim allows us to sample the output of a noisy stabilizer circuit, and to analyze the effect of noise on the stabilizer measurements. Just as before, these stabilizer measurements can then be fed into a decoder to correct the errors.

However, whereas before we had a very clear intuitive picture of matching as pairing up syndromes laid out on a 2D grid, the problem of matching syndromes in a repeated execution of a noisy circuit is quite a different thing. This can already be seen by just looking at the matching graph corresponding to the planar code error correction circuit:

In [ ]:
circuit.diagram("matchgraph-3d")

Although this 3D matching problem seems quite a bit more involved than the case we considered previously, the principle remains exactly the same. In particular, our matching decoder will suggest a correction operation, after which we can check if the product of the error and the correction operator corresponds to a non-trivial logical operator or not.

### Fault-tolerant threshold <a id='fault_tolerant'></a>

In [ ]:
import sinter

Given all these additional error processes that can occur in the circuit picture, we can ask how this affects the error correction threshold.

We can do this by repeating exactly the kind of simulation we did before, but now for the surface code under circuit noise. Here, we will take the noise rates for all the error processes to be equal, and we will vary this global noise rate to see how the logical failure rate for different code sizes. Again, we hope to find a critical value $p_{\text{th}}$ of the physical error rate below which the logical failure rate goes to zero in the limit of large code size. This threshold would be called 'fault-tolerant', since it persists in the setting where all aspects of the error correction procedure are noisy.

This time, we will perform the simulation according to the functionality provided by the [`sinter` sampling module](https://github.com/quantumlib/Stim/tree/main/glue/sample) built on top of Stim.

In [ ]:
surface_code_tasks = [
    sinter.Task(
        circuit = stim.Circuit.generated(
            "surface_code:rotated_memory_z",
            rounds=d * 3,
            distance=d,
            after_clifford_depolarization=noise,
            after_reset_flip_probability=noise,
            before_measure_flip_probability=noise,
            before_round_data_depolarization=noise,
        ),
        json_metadata={'d': d, 'r': d * 3, 'p': noise},
    )
    for d in [3, 5, 7]
    for noise in [0.008, 0.009, 0.01, 0.011, 0.012]
]

collected_surface_code_stats = sinter.collect(
    num_workers=4,
    tasks=surface_code_tasks,
    decoders=['pymatching'],
    max_shots=5_000_000,
    max_errors=10_000,
    print_progress=True,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats,
    x_func=lambda stat: stat.json_metadata['p'],
    group_func=lambda stat: stat.json_metadata['d'],
    failure_units_per_shot_func=lambda stat: stat.json_metadata['r'],
)
ax.set_ylim(5e-3, 5e-2)
ax.set_xlim(0.008, 0.012)
ax.loglog()
ax.set_title("Surface code simulation\n(Circuit noise, MWPM decoder")
ax.set_xlabel("Phyical error rate")
ax.set_ylabel("Logical failure rate per round")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120)

From this plot we can see that we find a fault-tolerant threshold of about $p_{\text{th}} \approx 1\%$. This means taking into account noise in the error correction circuits reduced the threshold by an order of magnitude. This, again, shows that dealing with errors in a realistic setting is a very difficult problem.